In [1]:
%cd mmdetection
import mmcv

/data/Detection_proj/mmdetection


# convert flag annotation to coco

In [2]:
import os.path as osp
import pdb
import json
def convert_flag_to_coco(ann_file, out_file, image_prefix):
    with open(ann_file) as f:
        data_infos = json.load(f)

    #data_infos = mmcv.load(ann_file)
    #pdb.set_trace()
    annotations = []
    images = []
    obj_count = 0
    for idx, v in enumerate(data_infos):
        filename = v['image_path']
        img_path = osp.join(image_prefix, filename)
        height, width = mmcv.imread(img_path).shape[:2]

        images.append(dict(
            id=idx,
            file_name=filename,
            height=height,
            width=width))

        bboxes = []
        labels = []
        masks = []
        for _, obj in enumerate(v['annotation']):
            #assert not obj['bbox']
            #obj = obj['shape_attributes']
            #px = obj['all_points_x']
            #py = obj['all_points_y']
            #poly = [(x + 0.5, y + 0.5) for x, y in zip(px, py)]
            #poly = [p for x in poly for p in x]

            #x_min, y_min, x_max, y_max = (
            #    min(px), min(py), max(px), max(py))
            x_min,y_min,x_max,y_max = obj['bbox'][0][0],obj['bbox'][0][1],obj['bbox'][1][0],obj['bbox'][1][1],
            
            if x_max-x_min < 0 or y_max-y_min < 0:
                print(v)
                
            
            category = int(obj['class_id'])

            data_anno = dict(
                image_id=idx,
                id=obj_count,
                category_id=category,
                bbox=[x_min, y_min, x_max - x_min, y_max - y_min],
                area=(x_max - x_min) * (y_max - y_min),
                segmentation=None,
                iscrowd=0)
            annotations.append(data_anno)
            obj_count += 1

    coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=[{'id':0, 'name': 'china'},
                    {'id':1, 'name': 'us'},
                   {'id':2, 'name': 'uk'},
                   {'id':3, 'name': 'russia'},
                   {'id':4, 'name': 'japan'},
                   {'id':5, 'name': 'france'},
                   {'id':6, 'name': 'german'},
                   {'id':7, 'name': 'italy'},
                   {'id':8, 'name': 'australia'},
                   {'id':9, 'name': 'korea'},
                   {'id':10, 'name': 'other'}])
    #mmcv.dump(coco_format_json, out_file)
    return coco_format_json

In [3]:
json_file = '/data/flag/flag_ann_train_all.json'
out_file = '/data/flag/flag_coco_train_all.json'
image_prefix = '/data/flag/Images'
flag_coco = convert_flag_to_coco(json_file, out_file, image_prefix)

In [4]:
flag_coco

{'images': [{'id': 0,
   'file_name': 'ffout-18/02039.jpg',
   'height': 360,
   'width': 640},
  {'id': 1, 'file_name': 'ffout-6/00726.jpg', 'height': 480, 'width': 852},
  {'id': 2, 'file_name': 'ffout-17/05885.jpg', 'height': 480, 'width': 852},
  {'id': 3, 'file_name': 'ffout-17/06284.jpg', 'height': 480, 'width': 852},
  {'id': 4, 'file_name': 'ffout-6/02971.jpg', 'height': 480, 'width': 852},
  {'id': 5, 'file_name': 'ffout-22/02530.jpg', 'height': 480, 'width': 854},
  {'id': 6, 'file_name': 'ffout-6/02248.jpg', 'height': 480, 'width': 852},
  {'id': 7, 'file_name': 'ffout-17/07614.jpg', 'height': 480, 'width': 852},
  {'id': 8, 'file_name': 'ffout-22/00886.jpg', 'height': 480, 'width': 854},
  {'id': 9, 'file_name': 'ffout-22/01078.jpg', 'height': 480, 'width': 854},
  {'id': 10, 'file_name': 'ffout-22/01721.jpg', 'height': 480, 'width': 854},
  {'id': 11, 'file_name': 'ffout-14/00153.jpg', 'height': 480, 'width': 852},
  {'id': 12, 'file_name': 'ffout-2/02931.jpg', 'height': 4

In [5]:
mmcv.dump(flag_coco, out_file)

# build model

In [3]:
from mmdet.apis import set_random_seed
from mmcv import Config
from pprint import pprint
#cfg = Config.fromfile('./configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_1x_coco.py')
#faster_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.378_20200504_180032-c5925ee5.pth
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_1x_coco.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='Cro

In [6]:
import os
import os.path as osp
CLASS = ['china','us','uk','russia','japan','france','german','italy','australia','korea','other']
                
# Modify dataset type and path
cfg.data_root = '/data/flag/Images'
cfg.classes = CLASS

cfg.data.test.img_prefix = '/data/flag/Images'
cfg.data.test.ann_file = '/data/flag/flag_coco_test.json'
cfg.data.test.classes = CLASS

cfg.data.train.ann_file = '/data/flag/flag_coco_train.json'
cfg.data.train.img_prefix = '/data/flag/Images'
cfg.data.train.classes = CLASS

cfg.data.val.img_prefix = '/data/flag/Images'
cfg.data.val.ann_file = '/data/flag/flag_coco_val.json'
cfg.data.val.classes = CLASS

# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 11
#cfg.model.roi_head.mask_head.num_classes = 1
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch
#cfg.load_from = 'checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_\
#coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
cfg.load_from = 'checkpoints/faster_rcnn_r50_caffe_fpn_1x_coco\
_bbox_mAP-0.378_20200504_180032-c5925ee5.pth'
# Set up working dir to save files and logs.
cfg.work_dir = './flag_proj/faster-rcnn'

if not os.path.exists(cfg.work_dir):
    os.makedirs(cfg.work_dir)
# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02 / 8
#cfg.lr_config.warmup = None
#log interval by step not epoch
cfg.log_config.interval = 100

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = ['bbox']
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.workflow = [('train',1),('val',1)]

#cfg.runner.max_epochs = 1

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')
cfg.dump(osp.join(cfg.work_dir, 'flag_config.py'))

Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=False),
        norm_eval=True,
        style='caffe',
        init_cfg=dict(
            type='Pretrained',
            checkpoint='open-mmlab://detectron2/resnet50_caffe')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='Cro

In [4]:
len(cfg.workflow)

2

In [7]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import os.path as osp
import copy

# Build dataset
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    val_dataset = copy.deepcopy(cfg.data.val)
    val_dataset.pipeline = cfg.data.train.pipeline
    datasets.append(build_dataset(val_dataset))
# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/data/Detection_proj/mmdetection/mmdet/core/anchor/builder.py:16: UserWarning: ``build_anchor_generator`` would be deprecated soon, please use ``build_prior_generator`` 
  '``build_anchor_generator`` would be deprecated soon, please use '


In [8]:
model

FasterRCNN(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): ResLayer(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [9]:
datasets

[
 CocoDataset Train dataset with number of images 2527, and instance counts: 
 +------------+-------+------------+-------+-----------+-------+---------------+-------+-----------+-------+
 | category   | count | category   | count | category  | count | category      | count | category  | count |
 +------------+-------+------------+-------+-----------+-------+---------------+-------+-----------+-------+
 | 0 [china]  | 2335  | 1 [us]     | 310   | 2 [uk]    | 349   | 3 [russia]    | 153   | 4 [japan] | 582   |
 | 5 [france] | 206   | 6 [german] | 180   | 7 [italy] | 161   | 8 [australia] | 66    | 9 [korea] | 947   |
 +------------+-------+------------+-------+-----------+-------+---------------+-------+-----------+-------+,
 
 CocoDataset Train dataset with number of images 361, and instance counts: 
 +------------+-------+------------+-------+-----------+-------+---------------+-------+-----------+-------+
 | category   | count | category   | count | category  | count | category      

In [ ]:
train_detector(model, datasets, cfg, distributed=False, validate=True)

2021-07-05 17:27:02,474 - mmdet - INFO - load checkpoint from checkpoints/faster_rcnn_r50_caffe_fpn_1x_coco_bbox_mAP-0.378_20200504_180032-c5925ee5.pth
2021-07-05 17:27:02,475 - mmdet - INFO - Use load_from_local loader
2021-07-05 17:27:02,649 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-07-05 17:27

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


/data/Detection_proj/mmdetection/mmdet/core/anchor/anchor_generator.py:323: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/data/Detection_proj/mmdetection/mmdet/core/anchor/anchor_generator.py:360: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '
2021-07-05 17:28:45,066 - mmdet - INFO - Epoch [1][100/1264]	lr: 4.970e-04, eta: 4:15:48, time: 1.019, data_time: 0.025, memory: 3609, loss_rpn_cls: 0.1084, loss_rpn_bbox: 0.0232, loss_cls: 0.6906, acc: 85.7676, loss_bbox: 0.1200, loss: 0.9423
2021-07-05 17:30:25,986 - mmdet - INFO - Epoch [1][200/1264]	lr: 9.965e-04, eta: 4:12:55, time: 1.009, data_time: 0.009, memory: 3609, loss_rpn_cls: 0.0426, loss_rpn_bbox: 0.0207, loss_cls: 0.1972, acc: 95.3164, loss_bbox: 0.1710, loss: 0.4316
2021-07-05 17:32:07,275 - mmdet - INFO - Ep

NameError: name 'workflow' is not defined